In [1]:
# libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy as sp

In [2]:
# original
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 25 21:16:14 2018

@author: T
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor 

#X = data.drop(['joke1'], axis = 1)
#y = data['joke1']
#first few variables of the first few principal components
#princomp = pd.DataFrame(pca.components_.T).iloc[:65,:1]
#top 10 features of first principal component
#pcafeatures = []
#for item in princomp.sort_values(by=[0], ascending=False).index.tolist()[:10]:
#    pcafeatures.append(X.columns[item])

def PCR(regr, labeling):
    pca = PCA()
    train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=100)
    
    X_reduced_train = pca.fit_transform(train_X)
    n = len(X_reduced_train)
    kf_10 = model_selection.KFold(n, n_folds=10, shuffle=True, random_state=1)
    r2 = []
    score = -1*cross_val_score(regr, np.ones((n,1)), train_y.ravel(), cv=kf_10, scoring='r2').mean()
    r2.append(score)
    
    for i in np.arange(1, 20):
        score = -1*model_selection.cross_val_score(regr, X_reduced_train[:,:i], train_y.ravel(), cv=kf_10, scoring='r2').mean()
        r2.append(score)
    
    plt.plot(r2, '-v', label=labeling)
    plt.xlabel('Number of principal components in regression')
    plt.ylabel('R2')
    plt.title('Predicting Joke 1')
    plt.xlim(xmin=-1);
    plt.legend()
    
    np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
    numcomp = r2.index(max(r2))
    print("Number of principal components: {}" .format(numcomp))
    
    X_reduced_test = pca.transform(test_X)[:,:numcomp+1]
    X_reduced_train = pca.transform(train_X)[:,:numcomp+1]
    regr.fit(X_reduced_train[:,:numcomp+1], train_y)
    pred_train = regr.predict(X_reduced_train)
    pred_test = regr.predict(X_reduced_test)
    print("{} train R2 : {:.3f}" .format(labeling, r2_score(train_y, pred_train)))
    print("{} test R2 : {:.3f}" .format(labeling, r2_score(test_y, pred_test)))
#    print("MSE: {:.3f}".format(mean_squared_error(test_y, pred)))
#    print('train R2 : {:.3f}'.format(regr.score(train_y, pred_train)))

def r2(pred, actual):
    RSS = 0
    TSS = 0
    for i in range(0, len(pred)):
        TSS = TSS + ((actual[i] - np.mean(actual))**2)
        RSS = RSS + ((actual[i] - pred[i])**2)
    R2 = 1-RSS/TSS
    return R2

def mse(y, y_pred):
    return np.mean((y-y_pred)**2)

def evaluation(regr, labeling, ranked_features):
    totaltrainr2 = []
    totaltestr2 = []
    totaltrainmse = []
    totaltestmse = []
    min_test_mse = 0
    max_test_r2 = 0
    num_features_to_use = 0
    
    #iterating over the ranked features from most to least important
    for i in range(1,len(ranked_features)):
        trainr2 = []
        testr2 = []
        mse_train = []
        mse_test = []
        test_features = ranked_features[:i]
        n = len(X[test_features])

        from sklearn.model_selection import KFold 
        kf = KFold(n_splits=7, shuffle=True) 
        
        for train_index, test_index in kf.split(X[test_features]):
              train_X, test_X = X[test_features].iloc[train_index], X[test_features].iloc[test_index] 
              train_y, test_y = y.iloc[train_index], y.iloc[test_index]     
             
              regr.fit(train_X, train_y)
              pred_train = regr.predict(train_X)
              pred_test = regr.predict(test_X)
              
              trainr2.append(r2(pred_train,list(train_y)))
              testr2.append(r2(pred_test, list(test_y)))              
              
              mse_train.append(mse(train_y, list(pred_train)))              
              mse_test.append(mse(test_y, list(pred_test)))   
          
        totaltrainr2.append(np.mean(trainr2))
        totaltestr2.append(np.mean(testr2))
        totaltrainmse.append(np.mean(mse_train))
        totaltestmse.append(np.mean(mse_test))
        
#        if min_test_mse == 0:
#            min_test_mse = np.mean(mse_test)
        if max_test_r2 == 0:
            max_test_r2 = np.mean(testr2)
#        if np.mean(mse_test) < min_test_mse:
        if np.mean(testr2) > max_test_r2:
#            min_test_mse = np.mean(mse_test)
#            min_train_mse = np.mean(mse_train)
            max_test_r2 = np.mean(testr2)
            max_train_r2 = np.mean(trainr2)
            num_of_features_to_use = i
            used_train_y = train_y
            used_pred_train = pred_train
            used_test_y = test_y
            used_pred_test = pred_test
            
    print("Number of features selected: {}".format(num_of_features_to_use))
    print("train R2 : {:.3f}" .format(max_train_r2))
    print("test R2 : {:.3f}" .format(max_test_r2))    
#    print("train MSE: {:.3f}".format(min_train_mse))
#    print("test MSE: {:.3f}".format(min_test_mse))
    
    #pred v. actual & residual plots        
    plt.scatter(used_train_y, used_pred_train, label='training set')
    plt.scatter(used_test_y, used_pred_test, label='test set')
    plt.plot( [-1,1],[-1,1] )
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title('Prediction plot for Joke 1')        
    plt.legend()
    plt.figure(figsize=(5,5))    

    plt.scatter(used_pred_train, used_pred_train-used_train_y, label='training set')
    plt.scatter(used_pred_test, used_pred_test-used_test_y, label='test set')
    plt.hlines(y=0, xmin=-1, xmax=1)
    plt.xlabel('Predicted')
    plt.ylabel('Residuals')
    plt.title('Residuals plot for Joke 1')
    plt.legend()
    plt.figure(figsize=(5,5))    

    #Training plots
#    plt.plot(totaltrainmse, '-v', label=labeling)
#    plt.xlabel('Number of features')
#    plt.ylabel('MSE training')
#    plt.title('Predicting Joke 1')
#    plt.xlim(xmin=-1);
#    plt.legend()
#    plt.figure(figsize=(5,5))

    plt.plot(totaltrainr2, '-v', label=labeling)
    plt.xlabel('Number of features')
    plt.ylabel('R2 training')
    plt.title('Predicting Joke 1')
    plt.xlim(xmin=-1);
    plt.legend()
    plt.figure(figsize=(5,5))

    #Testing plots
#    plt.plot(totaltestmse, '-v', label=labeling)
#    plt.xlabel('Number of features')
#    plt.ylabel('MSE testing')
#    plt.title('Predicting Joke 1')
#    plt.xlim(xmin=-1);
#    plt.legend()
#    plt.figure(figsize=(5,5))    
    
    plt.plot(totaltestr2, '-v', label=labeling)
    plt.xlabel('Number of features')
    plt.ylabel('R2 testing')
    plt.title('Predicting Joke 1')
    plt.xlim(xmin=-1);
    plt.legend()
    plt.figure(figsize=(5,5))    
    
    
def rank_to_dict(ranks, names, order=1):
    from sklearn.preprocessing import MinMaxScaler
    
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))

def list_features(key, features, names):
    featurelist = []
    for i in features.sort_values(key, ascending = False).index.values:
        featurelist.append(names[i])
    return(featurelist)         

def feature_selection():
    from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
    from sklearn.feature_selection import RFE, f_regression
     
    np.random.seed(0)
    names = X.columns
    ranks = {}

    lr = LinearRegression(fit_intercept=True,normalize=False)         
    lr.fit(X, y)
    ranks["Linear reg"] = rank_to_dict(np.abs(lr.coef_), names)
     
    ridge = Ridge(alpha=7)
    ridge.fit(X, y)
    ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)
    print ridge.coef_
     
    lasso = Lasso(alpha=.05)
    lasso.fit(X, y)
    ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)
      
    ##stop the search when 5 features are left (they will get equal scores)
    regr = RandomForestRegressor(n_estimators=50, max_features="sqrt", min_samples_leaf=4, max_depth=3, bootstrap=True)
    rfe = RFE(regr, n_features_to_select=5)
    rfe.fit(X,y)
    ranks["RFE"] = rank_to_dict(list(map(float,rfe.ranking_)), names, order=-1)
     
    rf = RandomForestRegressor(n_estimators=50, max_features="sqrt", min_samples_leaf=4, max_depth=3, bootstrap=True)
    rf.fit(X,y)
    ranks["RF"] = rank_to_dict(rf.feature_importances_, names)
     
    f, pval  = f_regression(X, y, center=True)
    ranks["Corr."] = rank_to_dict(f, names)
     
    r = {}
    for name in names:
        r[name] = round(np.mean([ranks[method][name] 
                                 for method in ranks.keys()]), 2)
    methods = sorted(ranks.keys())
    ranks["Mean"] = r
    methods.append("Mean")
    
    #extract features in order of most to least important
    matrix = []
    matrix.append(methods)
    for name in names:
        matrix.append(list(map(str,[ranks[method][name] for method in methods])))
    npmat = np.array(matrix)
    ranked_features = pd.DataFrame(data=npmat[1:,0:], columns=npmat[0,0:])

    #just testing random forest for now
    models = [RandomForestRegressor(n_estimators=50, max_features="sqrt", min_samples_leaf=4, max_depth=3, bootstrap=True)] #SVR(kernel="sigmoid", C=0.5, coef0=-0.5)] #MLPRegressor(hidden_layer_sizes = (3),max_iter=500)]    
    for model in models: 
        print(model)
        for feature_selection_method in methods:
            print(feature_selection_method)
#        feature_selection_method = "RFE" #Just testing RFE for now    
            evaluation(model, feature_selection_method, list_features(feature_selection_method,ranked_features,names))

    
def LOF(): #Removing outliers
    from sklearn.neighbors import LocalOutlierFactor
    
    clf = LocalOutlierFactor(n_neighbors=20)
    y_pred = clf.fit_predict(data)
    outliers = [i for i, x in enumerate(y_pred.tolist()) if x == -1]
    print(outliers)
    print(data)
    data.drop(outliers, inplace=True)
    data.reset_index(drop=True, inplace=True)
    data.to_csv("dataoutlier.csv")
    

data = pd.read_csv('data-LOFapplied.csv')
#LOF()
jokes = list(data.columns)[list(data.columns).index('joke1'):len(data.columns)]
X = data.drop(jokes[75], axis=1)
y = data[jokes[75]]
feature_selection()

[ 0.05616702 -0.00904002  0.08449317 -0.02617768 -0.02344046  0.00341648
 -0.02925148 -0.0769534   0.0416532   0.00465215  0.03080225 -0.00015419
 -0.02153258  0.06216178 -0.01217907 -0.01760239 -0.01084774 -0.01884328
  0.07868163 -0.02327956 -0.00729174 -0.06476905  0.05912341 -0.02318479
 -0.00539748 -0.01388242  0.03601099 -0.02961431 -0.04727906  0.03619413
 -0.00659561  0.04888683  0.01771703  0.01431113 -0.01746303 -0.01615712
  0.00280835  0.01161592  0.02696705 -0.03664062  0.00205354 -0.02827559
 -0.01635518  0.02243103  0.01792329  0.0263174   0.01234471  0.03100085
 -0.0382061  -0.00531016 -0.03121696  0.05207709 -0.00535944 -0.01314198
 -0.01069224  0.05785301 -0.03700406  0.02016743 -0.01377784 -0.00097837
 -0.05404643 -0.03335207 -0.01918162  0.08995958  0.01662054 -0.00612103
  0.06683142  0.01804851  0.00241752 -0.01921233  0.0621225   0.01944368
  0.03014305  0.02513669  0.02725447 -0.0314398   0.08582475 -0.03678468
  0.03955955  0.0251792   0.00409261 -0.03880875  0

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 25 21:16:14 2018

@author: T
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor 

#X = data.drop(['joke1'], axis = 1)
#y = data['joke1']
#first few variables of the first few principal components
#princomp = pd.DataFrame(pca.components_.T).iloc[:65,:1]
#top 10 features of first principal component
#pcafeatures = []
#for item in princomp.sort_values(by=[0], ascending=False).index.tolist()[:10]:
#    pcafeatures.append(X.columns[item])

def PCR(regr, labeling):
    pca = PCA()
    train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=100)
    
    X_reduced_train = pca.fit_transform(train_X)
    n = len(X_reduced_train)
    kf_10 = model_selection.KFold(n, n_folds=10, shuffle=True, random_state=1)
    r2 = []
    score = -1*cross_val_score(regr, np.ones((n,1)), train_y.ravel(), cv=kf_10, scoring='r2').mean()
    r2.append(score)
    
    for i in np.arange(1, 20):
        score = -1*model_selection.cross_val_score(regr, X_reduced_train[:,:i], train_y.ravel(), cv=kf_10, scoring='r2').mean()
        r2.append(score)
    
    plt.plot(r2, '-v', label=labeling)
    plt.xlabel('Number of principal components in regression')
    plt.ylabel('R2')
    plt.title('Predicting Joke 1')
    plt.xlim(xmin=-1);
    plt.legend()
    
    np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
    numcomp = r2.index(max(r2))
    print("Number of principal components: {}" .format(numcomp))
    
    X_reduced_test = pca.transform(test_X)[:,:numcomp+1]
    X_reduced_train = pca.transform(train_X)[:,:numcomp+1]
    regr.fit(X_reduced_train[:,:numcomp+1], train_y)
    pred_train = regr.predict(X_reduced_train)
    pred_test = regr.predict(X_reduced_test)
    print("{} train R2 : {:.3f}" .format(labeling, r2_score(train_y, pred_train)))
    print("{} test R2 : {:.3f}" .format(labeling, r2_score(test_y, pred_test)))
#    print("MSE: {:.3f}".format(mean_squared_error(test_y, pred)))
#    print('train R2 : {:.3f}'.format(regr.score(train_y, pred_train)))

def r2(pred, actual):
    RSS = 0
    TSS = 0
    for i in range(0, len(pred)):
        TSS = TSS + ((actual[i] - np.mean(actual))**2)
        RSS = RSS + ((actual[i] - pred[i])**2)
    R2 = 1-RSS/TSS
    return R2

def mse(y, y_pred):
    return np.mean((y-y_pred)**2)

def evaluation(regr, labeling, ranked_features):
    totaltrainr2 = []
    totaltestr2 = []
    totaltrainmse = []
    totaltestmse = []
    min_test_mse = 0
    max_test_r2 = 0
    num_features_to_use = 0
    
    #iterating over the ranked features from most to least important
    for i in range(1,len(ranked_features)):
        trainr2 = []
        testr2 = []
        mse_train = []
        mse_test = []
        test_features = ranked_features[:i]
        n = len(X[test_features])

        from sklearn.model_selection import KFold 
        kf = KFold(n_splits=7, shuffle=True) 
        
        for train_index, test_index in kf.split(X[test_features]):
              train_X, test_X = X[test_features].iloc[train_index], X[test_features].iloc[test_index] 
              train_y, test_y = y.iloc[train_index], y.iloc[test_index]     
             
              regr.fit(train_X, train_y)
              pred_train = regr.predict(train_X)
              pred_test = regr.predict(test_X)
              
              trainr2.append(r2(pred_train,list(train_y)))
              testr2.append(r2(pred_test, list(test_y)))              
              
              mse_train.append(mse(train_y, list(pred_train)))              
              mse_test.append(mse(test_y, list(pred_test)))   
          
        totaltrainr2.append(np.mean(trainr2))
        totaltestr2.append(np.mean(testr2))
        totaltrainmse.append(np.mean(mse_train))
        totaltestmse.append(np.mean(mse_test))
        
#        if min_test_mse == 0:
#            min_test_mse = np.mean(mse_test)
        if max_test_r2 == 0:
            max_test_r2 = np.mean(testr2)
#        if np.mean(mse_test) < min_test_mse:
        if np.mean(testr2) > max_test_r2:
#            min_test_mse = np.mean(mse_test)
#            min_train_mse = np.mean(mse_train)
            max_test_r2 = np.mean(testr2)
            max_train_r2 = np.mean(trainr2)
            num_of_features_to_use = i
            used_train_y = train_y
            used_pred_train = pred_train
            used_test_y = test_y
            used_pred_test = pred_test
            
    print("Number of features selected: {}".format(num_of_features_to_use))
    print("train R2 : {:.3f}" .format(max_train_r2))
    print("test R2 : {:.3f}" .format(max_test_r2))    
#    print("train MSE: {:.3f}".format(min_train_mse))
#    print("test MSE: {:.3f}".format(min_test_mse))
    
    #pred v. actual & residual plots        
    plt.scatter(used_train_y, used_pred_train, label='training set')
    plt.scatter(used_test_y, used_pred_test, label='test set')
    plt.plot( [-1,1],[-1,1] )
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title('Prediction plot for Joke 1')        
    plt.legend()
    plt.figure(figsize=(5,5))    

    plt.scatter(used_pred_train, used_pred_train-used_train_y, label='training set')
    plt.scatter(used_pred_test, used_pred_test-used_test_y, label='test set')
    plt.hlines(y=0, xmin=-1, xmax=1)
    plt.xlabel('Predicted')
    plt.ylabel('Residuals')
    plt.title('Residuals plot for Joke 1')
    plt.legend()
    plt.figure(figsize=(5,5))    

    #Training plots
#    plt.plot(totaltrainmse, '-v', label=labeling)
#    plt.xlabel('Number of features')
#    plt.ylabel('MSE training')
#    plt.title('Predicting Joke 1')
#    plt.xlim(xmin=-1);
#    plt.legend()
#    plt.figure(figsize=(5,5))

    plt.plot(totaltrainr2, '-v', label=labeling)
    plt.xlabel('Number of features')
    plt.ylabel('R2 training')
    plt.title('Predicting Joke 1')
    plt.xlim(xmin=-1);
    plt.legend()
    plt.figure(figsize=(5,5))

    #Testing plots
#    plt.plot(totaltestmse, '-v', label=labeling)
#    plt.xlabel('Number of features')
#    plt.ylabel('MSE testing')
#    plt.title('Predicting Joke 1')
#    plt.xlim(xmin=-1);
#    plt.legend()
#    plt.figure(figsize=(5,5))    
    
    plt.plot(totaltestr2, '-v', label=labeling)
    plt.xlabel('Number of features')
    plt.ylabel('R2 testing')
    plt.title('Predicting Joke 1')
    plt.xlim(xmin=-1);
    plt.legend()
    plt.figure(figsize=(5,5))    
    
    
def rank_to_dict(ranks, names, order=1):
    from sklearn.preprocessing import MinMaxScaler
    
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))

def list_features(key, features, names):
    featurelist = []
    for i in features.sort_values(key, ascending = False).index.values:
        featurelist.append(names[i])
    return(featurelist)         

def feature_selection():
    from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
    from sklearn.feature_selection import RFE, f_regression
     
    np.random.seed(0)
    names = X.columns
    ranks = {}

    lr = LinearRegression(fit_intercept=True,normalize=False)         
    lr.fit(X, y)
    ranks["Linear reg"] = rank_to_dict(np.abs(lr.coef_), names)

     
    ridge = Ridge(alpha=7)
    ridge.fit(X, y)
    ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)
    lasso = Lasso(alpha=.05)
    lasso.fit(X, y)
    ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)
    print lasso.coef_
    ##stop the search when 5 features are left (they will get equal scores)
    regr = RandomForestRegressor(n_estimators=50, max_features="sqrt", min_samples_leaf=4, max_depth=3, bootstrap=True)
    rfe = RFE(regr, n_features_to_select=5)
    rfe.fit(X,y)
    ranks["RFE"] = rank_to_dict(list(map(float,rfe.ranking_)), names, order=-1)
     
    rf = RandomForestRegressor(n_estimators=50, max_features="sqrt", min_samples_leaf=4, max_depth=3, bootstrap=True)
    rf.fit(X,y)
    ranks["RF"] = rank_to_dict(rf.feature_importances_, names)
     
    f, pval  = f_regression(X, y, center=True)
    ranks["Corr."] = rank_to_dict(f, names)
     
    r = {}
    for name in names:
        r[name] = round(np.mean([ranks[method][name] 
                                 for method in ranks.keys()]), 2)
    methods = sorted(ranks.keys())
    ranks["Mean"] = r
    methods.append("Mean")
    
    #extract features in order of most to least important
    matrix = []
    matrix.append(methods)
    for name in names:
        matrix.append(list(map(str,[ranks[method][name] for method in methods])))
    npmat = np.array(matrix)
    ranked_features = pd.DataFrame(data=npmat[1:,0:], columns=npmat[0,0:])

    #just testing random forest for now
    models = [RandomForestRegressor(n_estimators=50, max_features="sqrt", min_samples_leaf=4, max_depth=3, bootstrap=True)] #SVR(kernel="sigmoid", C=0.5, coef0=-0.5)] #MLPRegressor(hidden_layer_sizes = (3),max_iter=500)]    
    for model in models: 
        print(model)
        for feature_selection_method in methods:
            print(feature_selection_method)
#        feature_selection_method = "RFE" #Just testing RFE for now    
            evaluation(model, feature_selection_method, list_features(feature_selection_method,ranked_features,names))

    
def LOF(): #Removing outliers
    from sklearn.neighbors import LocalOutlierFactor
    
    clf = LocalOutlierFactor(n_neighbors=20)
    y_pred = clf.fit_predict(data)
    outliers = [i for i, x in enumerate(y_pred.tolist()) if x == -1]
    print(outliers)
    print(data)
    data.drop(outliers, inplace=True)
    data.reset_index(drop=True, inplace=True)
    data.to_csv("dataoutlier.csv")
    

data = pd.read_csv('data-LOFapplied.csv')
#LOF()
jokes = list(data.columns)[list(data.columns).index('joke1'):len(data.columns)]
X = data.drop(jokes[75], axis=1)
y = data[jokes[75]]
feature_selection()

In [32]:
data = pd.read_csv('data-LOFapplied.csv')
#LOF()
jokes = list(data.columns)[list(data.columns).index('joke1'):len(data.columns)]
X = data.drop(jokes[75], axis=1)
y = data[jokes[75]]

,Gender Role,Region of Ancestry: Africa,Region of Ancestry: East Asia and the Pacific,Region of Ancestry: Europe and Central Asia,Region of Ancestry: South Asia,Region of Ancestry: Middle East and North Africa,Region of Ancestry: Latin America and The Carribean,Current region of occupancy: West Coast,Current region of occupancy: The Midwest,Current region of occupancy: East Coast,...,joke112,joke113,joke114,joke115,joke116,joke117,joke118,joke119,joke120,joke121
0,0.1615,1,0,0,0,0,0,1,0,0,...,-1.00,-1.00,-1.00,0.01,-1.00,-1.00,-0.52,-1.00,-1.00,-1.00
1,0.4720,0,1,0,0,0,0,1,0,0,...,-0.84,-0.64,-0.97,-0.91,-0.56,-0.99,0.51,-0.99,-0.79,-0.76
2,0.5475,0,0,1,0,0,0,0,1,0,...,0.27,-1.00,-0.26,0.69,0.30,-0.77,0.49,-0.72,0.31,0.19
3,0.3385,0,0,1,0,0,0,1,0,0,...,0.24,-0.74,-0.97,0.92,0.29,-0.95,-0.92,-0.86,0.56,0.20
4,0.2310,0,0,0,1,0,0,0,1,0,...,-0.25,-0.76,-0.82,0.89,0.49,-1.00,0.27,-0.89,-0.60,-0.36
5,0.1340,0,0,1,0,0,0,1,0,0,...,-0.92,-0.84,-0.83,-0.78,-0.71,0.48,-0.90,-0.95,-0.85,-0.78
6,0.5425,1,0,0,0,0,0,0,0,1,...,0.11,-0.97,-0.29,0.76,0.51,-0.99,0.20,-0.89,0.48,0.13
7,0.4160,0,0,1,0,0,0,0,1,0,...,-0.96,-0.97,-0.98,0.19,-0.37,-0.96,0.20,-0.58,0.00,0.13
8,0.4620,0,0,1,0,0,0,0,0,1,...,-0.92,-1.00,-1.00,-0.83,-0.53,0.54,-0.97,0.78,-0.88,0.72
9,0.3595,0,0,1,0,0,0,0,1,0,...,0.32,0.12,0.14,0.41,0.14,-0.07,0.19,0.00,0.05,0.09


In [51]:
class LinearRegression:
    def __init__(self, max_iter=1000):
        self.coef_ = 0.0
        self.intercept_ = 0.0
        self.max_iter_ = max_iter

    def fit(self, X, y):
        # Gradient Descent
        X = np.array(X)
        y = np.array(y)
        
        rows = X.shape[0]
        n_dim = X.shape[1]

        tf_X = tf.placeholder(tf.float32, [n_dim])
        tf_y = tf.placeholder(tf.float32)

        # set random weight
        W = tf.Variable([0. for i in range(n_dim)], name="weight")
        b = tf.Variable(0., name="bias")
        learning_rate = 0.01
        epochs = self.max_iter_

        # Construct a linear model
        mult = tf.tensordot(tf_X, W, 1)
        pred = tf.add(mult, b)
        cost = ((pred-tf_y)** 2)/(2.0*rows)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
        init = tf.global_variables_initializer()

        # start training
        with tf.Session() as sess:
            sess.run(init)

            for epoch in range(epochs):
                for _x, _y in zip(X, y):
                    sess.run(optimizer, feed_dict={tf_X: _x, tf_y: _y})

            print ("Optimization Finished")
            self.coef_ = sess.run(W)
            self.intercept = sess.run(b)
            
    def predict(self, X):
        result = []
        X = np.array(X)
        for row in X:
            result.append(np.dot(self.coef_, row.T) + self.intercept_)
            
        return result

    
model = LinearRegression()
model.fit(X, y)
pred = model.predict(X)

from sklearn.metrics import mean_squared_error, r2_score
print model.coef_
print mean_squared_error(pred, y)
print r2_score(pred, y)

# names = X.columns
# ranks = {}
# ranks["Linear reg"] = rank_to_dict(np.abs(linear_regression(X, y)), names)

Optimization Finished
[ 0.0563402  -0.00254857  0.08365893 -0.02077642 -0.01527573  0.00192926
 -0.02588599 -0.0756564   0.0365946   0.01872241  0.04225377 -0.00081299
 -0.00780984  0.06267536 -0.00255291 -0.01996913 -0.01124196 -0.0169574
  0.08656354 -0.02734428 -0.00396399 -0.06999615  0.0665854  -0.01629515
 -0.00554533 -0.00890307  0.0288071  -0.02523275 -0.04967061  0.03402118
 -0.00021457  0.05665257  0.02077402  0.01439209 -0.01421239 -0.01540838
  0.002817    0.00362212  0.02600806 -0.04225134  0.02253742 -0.02203666
 -0.02118871  0.01664868  0.03252444  0.03728252  0.00182658  0.02285272
 -0.02538225  0.00032582 -0.02467646  0.04836565 -0.00336085 -0.0027534
 -0.01103387  0.04700733 -0.03024316  0.02108433 -0.00797086 -0.00247315
 -0.03704345 -0.03522637 -0.00924647  0.07990892  0.02270892 -0.00896773
  0.074018    0.01135668  0.00898364 -0.02257344  0.07141442  0.03000204
  0.02949349  0.03829817  0.03183754 -0.0320742   0.08965413 -0.05658497
  0.04592396  0.03300231 -0.005

In [6]:
import numpy as np
from copy import deepcopy

class Lasso():
    def __init__(self, alpha=1.0, max_iter=1000, fit_intercept=True):
        self.alpha = alpha
        self.max_iter = max_iter
        self.fit_intercept = fit_intercept
        self.coef_ = None
        self.intercept_ = None

    def _soft_thresholding_operator(self, x, lambda_):
        if x > 0 and lambda_ < abs(x):
            return x - lambda_
        elif x < 0 and lambda_ < abs(x):
            return x + lambda_
        else:
            return 0

    def fit(self, X, y):
        if self.fit_intercept:
            X = np.column_stack((np.ones(len(X)),X))

        beta = np.zeros(X.shape[1])
        if self.fit_intercept:
            beta[0] = np.sum(y - np.dot(X[:, 1:], beta[1:]))/(X.shape[0])

        for iteration in range(self.max_iter):
            start = 1 if self.fit_intercept else 0
            for j in range(start, len(beta)):
                tmp_beta = deepcopy(beta)
                tmp_beta[j] = 0.0
                r_j = y - np.dot(X, tmp_beta)
                arg1 = np.dot(X[:, j], r_j)
                arg2 = self.alpha*X.shape[0]

                beta[j] = self._soft_thresholding_operator(arg1, arg2)/(X[:, j]**2).sum()

            if self.fit_intercept:
                beta[0] = np.sum(y - np.dot(X[:, 1:], beta[1:]))/(X.shape[0])

        if self.fit_intercept:
            self.intercept_ = beta[0]
            self.coef_ = beta[1:]
        else:
            self.coef_ = beta

        return self

    def predict(self, X):
        y = np.dot(X, self.coef_)
        if self.fit_intercept:
            y += self.intercept_*np.ones(len(y))
        return y

model = Lasso(alpha=0.05, max_iter=1000)
model.fit(X, y)
print model.coef_

[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00

In [64]:
class Ridge:
    def __init__(self, alpha=1.0, max_iter=1000):
        self.coef_ = 0.0
        self.intercept_ = 0.0
        self.alpha = alpha
        self.max_iter = max_iter

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        rows = X.shape[0]
        n_dim = X.shape[1]

        tf_X = tf.placeholder(tf.float32, [n_dim])
        tf_y = tf.placeholder(tf.float32)

        # set random weight
        W = tf.Variable([.1 for i in range(n_dim)], name="weight")
        b = tf.Variable(.1, name="bias")
        learning_rate = 0.01
        epochs = self.max_iter

        # Construct a linear model
        mult = tf.tensordot(tf_X, W, 1)
        pred = tf.add(mult, b)
        penalty = self.alpha * (tf.norm(W)**2)
        rss = ((pred-tf_y)**2) / (2.0*rows)
        cost = rss + penalty
        
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
        init = tf.global_variables_initializer()

        # start training
        with tf.Session() as sess:
            sess.run(init)

            for epoch in range(epochs):
                for _x, _y in zip(X, y):
                    sess.run(optimizer, feed_dict={tf_X: _x, tf_y: _y})

            print ("Optimization Finished")
            self.coef_ = sess.run(W)
            self.intercept_ = sess.run(b)
    
    def predict(self, X):
        result = []
        X = np.array(X)
        for row in X:
            result.append(np.dot(self.coef_, row.T) + self.intercept_)
            
        return result
    
ridge = Ridge(7., 2000)
ridge.fit(X, y)

print ridge.coef_
print ridge.intercept_
from sklearn.metrics import mean_squared_error, r2_score
pred = ridge.predict(X)
print mean_squared_error(pred, y)
print r2_score(pred, y)

Optimization Finished
[  4.31204389e-05  -4.22686753e-06   1.01093974e-05  -3.81816208e-05
   1.66220379e-05   9.13766871e-06   3.00869061e-07  -2.40450045e-05
   9.24647611e-05  -1.69751293e-05  -5.84689697e-05   7.85833493e-07
  -4.22780158e-06  -3.76719108e-05   4.44673497e-05  -8.74372219e-08
  -8.71870634e-06   4.46425547e-05   2.32461807e-05   3.35954865e-05
   5.04502968e-05  -1.10750931e-04  -1.59632577e-06  -1.16439912e-06
  -1.74326869e-05   1.74138695e-05   1.86117133e-04  -3.42412960e-07
   7.33339330e-05  -5.27856264e-06   1.32837813e-05   6.01443608e-05
  -2.00366430e-05  -1.27547872e-04   5.27808979e-08   1.52122951e-07
   2.15602222e-06   1.34725080e-04   1.79846811e-05  -7.94950756e-05
   7.32957142e-06  -1.01203004e-04  -3.19426836e-05  -1.08422304e-04
  -8.61466833e-05   1.15099865e-04  -9.85739462e-06  -2.15362725e-05
   1.02307524e-04  -1.91444542e-05  -6.69737492e-05  -4.02334672e-06
   3.30698458e-06   7.42172879e-10  -1.16346416e-06   1.43010209e-06
  -4.4256944